# Link Google Drive data and load required libraries

Mount Google Drive to access files


In [0]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir('/content/gdrive/My Drive/LLR_NN')
!ls
!pip install tensorflow-gpu==2.1.0

Import Required Libaries

In [0]:
import numpy as np
import scipy.io as sio
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, MaxPooling1D
from tensorflow.keras.layers import Activation,Conv2D, Flatten, Permute, Cropping2D
from tensorflow.keras.layers import ZeroPadding2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Reshape
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
import matplotlib.pyplot as plt

Load Data

In [5]:
MODULATION = "BPSK"
NOISE_VAR = "1e-2"

# Load the training data and split into validation set as well
X_train_valid = data = np.genfromtxt(MODULATION + "_X_train_var_" + NOISE_VAR + ".csv", delimiter=',')
y_train_valid = np.genfromtxt(MODULATION + "_X_train_var_" + NOISE_VAR + ".csv", delimiter=',')
X_train, X_val, y_train, y_val= train_test_split(X_train_valid, y_train_valid, test_size=0.15)

# Load the test data
X_test = np.genfromtxt(MODULATION + "_X_train_var_" + NOISE_VAR + ".csv", delimiter=',')
y_test = np.genfromtxt(MODULATION + "_X_train_var_" + NOISE_VAR + ".csv", delimiter=',')

OSError: ignored

# Construct the Neural Network
